In [33]:
import pandas as pd

# URLs
url1 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_add_on_categories_cleaned.csv"
url2 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_add_ons_cleaned.csv"
url3 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_items_cleaned.csv"
url4 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_item_add_on_categories_cleaned.csv"
url5 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_item_add_ons_cleaned.csv"
url6 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_items_cleaned.csv"
url7 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_menu_sections_cleaned.csv"
url8 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_sections_cleaned.csv"
# FIXED: Removed the extra dot after 'raw'
url9 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/dim_users_cleaned.csv"
url10 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_desc_cleaned.csv"

url11 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_dogs.csv"

url12 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_input_items.csv"

url13 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis%20/Datasets/menu_engineering_plowhorse_whatif.csv"
# Loading Data
dim_add_on_categories_cleaned = pd.read_csv(url1)
dim_add_ons_cleaned = pd.read_csv(url2)
dim_items_cleaned = pd.read_csv(url3, low_memory=False)
dim_menu_item_add_on_categories_cleaned = pd.read_csv(url4)
dim_menu_item_add_ons_cleaned = pd.read_csv(url5)
dim_menu_items_cleaned = pd.read_csv(url6)
dim_menu_sections_cleaned = pd.read_csv(url7)
dim_sections_cleaned = pd.read_csv(url8)
dim_users_cleaned = pd.read_csv(url9, low_memory=False)
analysis1  = pd.read_csv(url10)
analysis2  = pd.read_csv(url11)
analysis3  = pd.read_csv(url12)
analysis4  = pd.read_csv(url13)



print(df.head())
print(df.info())

      id  section_id     created         title    type  status  rating  votes  \
0  59911       72091  1611005512     Artichoke  Normal  Active      67      7   
1  59912       72091  1611005528    Broccolien  Normal  Active      30      3   
2  59913       72091  1611005553        Potato  Normal  Active      18      2   
3  59916       72091  1611005610      Mushroom  Normal  Active      47      5   
4  59917       72091  1611005630  Sunny Hawaii  Normal  Active     102     11   

   purchases  price  index     created_datetime  revenue  estimated_cost  \
0        107  109.0      0  2021-01-18T23:31:52  11663.0            43.6   
1         31  129.0      1  2021-01-18T23:32:08   3999.0            51.6   
2        103  129.0      2  2021-01-18T23:32:33  13287.0            51.6   
3         73  139.0      5  2021-01-18T23:33:30  10147.0            55.6   
4         69  129.0      6  2021-01-18T23:33:50   8901.0            51.6   

   margin menu_label  
0    65.4       Star  
1    77.4 

In [34]:
# Assuming df_model is the dataframe we created earlier
def calculate_item_confidence(df):
    # Normalize values between 0 and 1 for comparison
    df['price_norm'] = (df['price'] - df['price'].min()) / (df['price'].max() - df['price'].min())
    df['margin_norm'] = (df['Margin'] - df['Margin'].min()) / (df['Margin'].max() - df['Margin'].min())

    # Calculate Confidence Score (Weighted: 60% Margin Stability, 40% Popularity)
    # This measures how 'confident' we are that this item is a high performer
    df['Supporter_Confidence'] = (df['margin_norm'] * 0.6) + (df['popularity_score'] * 0.4)

    # Convert to Percentage
    df['Supporter_Confidence'] = df['Supporter_Confidence'] * 100
    return df

df_model = calculate_item_confidence(df_model)
print(df_model[['item_id', 'Category', 'Supporter_Confidence']].head())

KeyError: 'price'

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# --------------------------------------------------
#  LOAD DATA (ONLY WHAT ACTUALLY EXISTS)
# --------------------------------------------------

url_menu_items = (
    "https://raw.githubusercontent.com/Radwa-Ayman239/"
    "MenueEngineering/refs/heads/main/"
    "Data%20Analysis%20/Datasets/dim_menu_items_cleaned.csv"
)

df = pd.read_csv(url_menu_items)

print("Columns in dataset:")
print(df.columns.tolist())

# --------------------------------------------------
#  BASIC CLEANING
# --------------------------------------------------

# Required columns for menu engineering
required_cols = ["price", "purchases"]

df = df.dropna(subset=required_cols)

# Ensure numeric
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["purchases"] = pd.to_numeric(df["purchases"], errors="coerce")

df = df.dropna(subset=required_cols)

# --------------------------------------------------
#  FEATURE ENGINEERING (BUSINESS-ALIGNED)
# --------------------------------------------------

# Revenue
df["revenue"] = df["price"] * df["purchases"]

# Estimated cost (40% food cost assumption)
df["estimated_cost"] = df["price"] * 0.40

# Contribution margin
df["margin"] = df["price"] - df["estimated_cost"]

# --------------------------------------------------
# CREATE MENU ENGINEERING LABELS (GROUND TRUTH)
# --------------------------------------------------

popularity_threshold = df["purchases"].median()
profit_threshold = df["margin"].median()

def menu_engineering_label(row):
    if row["purchases"] >= popularity_threshold and row["margin"] >= profit_threshold:
        return "Star"
    elif row["purchases"] >= popularity_threshold and row["margin"] < profit_threshold:
        return "Plowhorse"
    elif row["purchases"] < popularity_threshold and row["margin"] >= profit_threshold:
        return "Puzzle"
    else:
        return "Dog"

df["menu_label"] = df.apply(menu_engineering_label, axis=1)

print("\nMenu category distribution:")
print(df["menu_label"].value_counts())

# --------------------------------------------------
# 5️⃣ PREPARE DATA FOR CLASSIFICATION MODEL
# --------------------------------------------------

features = ["price", "purchases", "margin"]
X = df[features]
y = df["menu_label"]

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

# --------------------------------------------------
# ⃣ TRAIN CLASSIFICATION MODEL
# --------------------------------------------------

model = DecisionTreeClassifier(
    max_depth=4,
    random_state=42
)

model.fit(X_train, y_train)

#  EVALUATION
y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=label_encoder.classes_
))




Columns in dataset:
['id', 'section_id', 'created', 'title', 'type', 'status', 'rating', 'votes', 'purchases', 'price', 'index', 'created_datetime']

Menu category distribution:
menu_label
Star         8303
Plowhorse    8181
Puzzle       7453
Dog          6470
Name: count, dtype: int64

Classification Report:
              precision    recall  f1-score   support

         Dog       1.00      1.00      1.00      1294
   Plowhorse       1.00      1.00      1.00      1636
      Puzzle       1.00      1.00      1.00      1491
        Star       1.00      1.00      1.00      1661

    accuracy                           1.00      6082
   macro avg       1.00      1.00      1.00      6082
weighted avg       1.00      1.00      1.00      6082

